In [2]:
import numpy as np

# Model A
![](ModelA.png)

In [22]:
L,M1,M2,M3,M4,I1,I2,I3,C,R,T=tuple(range(11))

In [23]:
#folded probabilities
A = np.zeros((11,11))
B = np.zeros((11,3))

In [24]:
S_folded = np.zeros(11)
S_folded[M1] = 0.6
S_folded[M2] = 0.1+0.1*0.8
S_folded[M3] = 0.1+0.1*0.2*0.5
S_folded[M4] = 0.1+0.1*0.2*0.5*0.8
S_folded[C] = 0.1*0.2*0.5*0.2*0.2
S_folded[R] = 0.1*0.2*0.5*0.2*0.7
S_folded[T] = 0.1*0.2*0.5*0.2*0.1

In [25]:
init = 0.5 * S_folded
init[L] = 0.5

In [26]:
A[L,:] = 0.4 * S_folded
A[L,L] = 0.6
A[M1,M2] = 0.97
A[M1,M3] = 0.01*0.5
A[M1,M4] = 0.01*0.5*0.8
A[M1,I1] = 0.01
M1_E = 0.01*0.5*0.2 + 0.01
A[M1,C] = 0.2 * M1_E
A[M1,R] = 0.7 * M1_E
A[M1,T] = 0.1 * M1_E
A[M2,M3] = 0.5
A[M2,M4] = 0.4*0.8
A[M2,I2] = 0.05
M2_E = 0.4*0.2 + 0.05
A[M2,C] = 0.2 * M2_E
A[M2,R] = 0.7 * M2_E
A[M2,T] = 0.1 * M2_E
A[M3,M4] = 0.6
A[M3,I3] = 0.3
M3_E = 0.05*2
A[M3,C] = 0.2 * M3_E
A[M3,R] = 0.7 * M3_E
A[M3,T] = 0.1 * M3_E
A[M4,C] = 0.2
A[M4,R] = 0.7
A[M4,T] = 0.1
A[I1,I1] = 0.5
A[I1,M2] = 0.5
A[I2,I2] = 0.5
A[I2,M3] = 0.5
A[I3,I3] = 0.5
A[I3,M4] = 0.5
A[C,:] = 0.1 * S_folded
A[C,C] = 0.9 + 0.1*0.1*0.2*0.5*0.2*0.2
A[R,R] = 0.6
A[R,T] = 0.4
A[T,T] = 1

In [27]:
B[:,:2] = 0.5
B[M2,:2] = (0.1, 0.9)
B[M3,:2] = (0.7, 0.3)
B[M4,:2] = (0.9, 0.1)
B[T,:] = (0,0,1)

In [28]:
np.testing.assert_almost_equal(np.sum(init), 1)
np.testing.assert_almost_equal(np.sum(A, -1), 1)
np.testing.assert_almost_equal(np.sum(B, -1), 1)

In [29]:
def forward(seq, init, A, B, scaled=False):
    num_states = init.shape[0]
    length = seq.shape[0]
    alpha = np.zeros((length, num_states))
    alpha[0] = init * B.dot(seq[0])
    if scaled:
        lik = np.sum(alpha[0])
        alpha[0] /= lik
    for i in range(1, length):
        alpha[i] = np.transpose(A).dot(alpha[i-1]) * B.dot(seq[i])
        if scaled:
            S = np.sum(alpha[i])
            alpha[i] /= S
            lik *= S
    if scaled:
        return alpha, lik
    else:
        return alpha

def backward(seq, init, A, B):
    num_states = init.shape[0]
    length = seq.shape[0]
    beta = np.zeros((length, num_states))
    beta[-1] = 1
    for i in range(1, length):
        b = B.dot(seq[length-i]) * beta[length-i]
        beta[length-1-i] = A.dot(b)
    return beta

def likelihood_forward(alpha):
    return np.sum(alpha[-1])

def likelihood_backward(beta, seq, init, B):
    return np.sum(init * B.dot(seq[0]) * beta[0])

def posterior(alpha, beta, lik):
    return alpha * beta / lik

In [30]:
seq = [0,1,0,2]
alphabet_size = B.shape[-1]
seq = np.eye(alphabet_size)[seq]

In [31]:
alpha = forward(seq, init, A, B)
beta = backward(seq, init, A, B)
lik = likelihood_forward(alpha)
lik2 = likelihood_backward(beta, seq, init, B)
np.testing.assert_almost_equal(lik, lik2)
posterior_probs = posterior(alpha, beta, lik)
alpha_scaled, lik3 = forward(seq, init, A, B, scaled=True)
np.testing.assert_almost_equal(lik, lik3)
print("alpha=\n", alpha)
print("beta=\n", beta)
print("lik=\n", lik)
print("posterior_probs=\n", posterior_probs)
print("alpha_scaled=\n", alpha_scaled)

alpha=
 [[2.50000000e-01 1.50000000e-01 9.00000000e-03 3.85000000e-02
  4.86000000e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e-04 3.50000000e-04 0.00000000e+00]
 [7.50000000e-02 3.00030000e-02 1.47151620e-01 4.87533000e-03
  3.73810800e-03 7.50000000e-04 2.25000000e-04 5.77500000e-03
  5.59200200e-03 1.95195070e-02 0.00000000e+00]
 [2.25000000e-02 9.16776006e-03 3.49785660e-03 5.40398859e-02
  5.06894598e-02 3.37515000e-04 3.73504050e-03 2.17504950e-03
  4.89105120e-03 1.41671282e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.13336506e-02]]
beta=
 [[6.55932264e-03 4.47112347e-02 2.02424452e-02 1.93615440e-02
  4.21142400e-02 2.28876170e-02 1.10750300e-02 1.82501000e-02
  1.81209072e-03 3.60000000e-02 0.00000000e+00]
 [4.55760160e-03 3.19602200e-03 5.05002600e-02 6.80002000e-02
  1.40002000e-01 6.50000000e-04 3.50000000e-03 4.50000000e-02


# Model B
![](ModelB.png)

In [32]:
L,M1,M2,M3,I1,I2,C,R,T=tuple(range(9))

In [33]:
#folded probabilities
A = np.zeros((9,9))
B = np.zeros((9,3))

In [34]:
S_folded = np.zeros(9)
S_folded[M1] = 0.7
S_folded[M2] = 0.1+0.1*0.8
S_folded[M3] = 0.1+0.1*0.2*0.5
S_folded[C] = 0.1*0.2*0.5*0.2
S_folded[R] = 0.1*0.2*0.5*0.7
S_folded[T] = 0.1*0.2*0.5*0.1

In [35]:
init = 0.5 * S_folded
init[L] = 0.5

In [36]:
A[L,:] = 0.4 * S_folded
A[L,L] = 0.6
A[M1,M2] = 0.97
A[M1,M3] = 0.01*0.5
A[M1,I1] = 0.01
M1_E = 0.01*0.5 + 0.01
A[M1,C] = 0.2 * M1_E
A[M1,R] = 0.7 * M1_E
A[M1,T] = 0.1 * M1_E
A[M2,M3] = 0.5
A[M2,I2] = 0.05
M2_E = 0.4 + 0.05
A[M2,C] = 0.2 * M2_E
A[M2,R] = 0.7 * M2_E
A[M2,T] = 0.1 * M2_E
A[M3,C] = 0.2 
A[M3,R] = 0.7 
A[M3,T] = 0.1 
A[I1,I1] = 0.5
A[I1,M2] = 0.5
A[I2,I2] = 0.5
A[I2,M3] = 0.5
A[C,:] = 0.1 * S_folded
A[C,C] = 0.9 + 0.1*0.1*0.2*0.5*0.2
A[R,R] = 0.6
A[R,T] = 0.4
A[T,T] = 1

In [37]:
B[:,:2] = 0.5
B[M2,:2] = (0.1, 0.9)
B[M3,:2] = (0.7, 0.3)
B[T,:] = (0,0,1)

In [38]:
np.testing.assert_almost_equal(np.sum(init), 1)
np.testing.assert_almost_equal(np.sum(A, -1), 1)
np.testing.assert_almost_equal(np.sum(B, -1), 1)

In [39]:
alpha = forward(seq, init, A, B)
beta = backward(seq, init, A, B)
lik = likelihood_forward(alpha)
lik2 = likelihood_backward(beta, seq, init, B)
np.testing.assert_almost_equal(lik, lik2)
posterior_probs = posterior(alpha, beta, lik)
alpha_scaled, lik3 = forward(seq, init, A, B, scaled=True)
np.testing.assert_almost_equal(lik, lik3)
print("alpha=\n", alpha)
print("beta=\n", beta)
print("lik=\n", lik)
print("posterior_probs=\n", posterior_probs)
print("alpha_scaled=\n", alpha_scaled)

alpha=
 [[2.50000000e-01 1.75000000e-01 9.00000000e-03 3.85000000e-02
  0.00000000e+00 0.00000000e+00 5.00000000e-04 1.75000000e-03
  0.00000000e+00]
 [7.50000000e-02 3.50175000e-02 1.68983100e-01 4.91415000e-03
  8.75000000e-04 2.25000000e-04 4.84255000e-03 1.66864250e-02
  0.00000000e+00]
 [2.25000000e-02 1.06694892e-02 3.98916409e-03 6.16926839e-02
  3.93837500e-04 4.28082750e-03 1.03578125e-02 3.36312550e-02
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.98273228e-02]]
beta=
 [[1.06095920e-02 8.64108263e-02 4.08254829e-02 4.21088510e-02
  4.46645250e-02 2.97515000e-02 2.82017450e-03 3.60000000e-02
  0.00000000e+00]
 [4.29404000e-03 6.81515000e-03 9.80045000e-02 1.40010000e-01
  2.25000000e-03 3.50000000e-02 1.08851000e-03 1.20000000e-01
  0.00000000e+00]
 [4.00000000e-04 1.50000000e-03 4.50000000e-02 1.00000000e-01
  0.00000000e+00 0.00000000e+00 1.00000000e-04 4.00000000e-01
  1.0000